In [237]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from lightfm import LightFM
from scipy.sparse import coo_matrix

In [238]:
data=pd.read_csv(r'./dataset/rating.csv')


In [239]:
data.head()


,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [242]:
data.shape

(20000263, 4)

In [243]:
df=data.copy()


In [244]:
df.shape

(20000263, 4)

In [245]:
users = df.userId.unique()
movies = df.movieId.unique()

userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

df['userId'] = df['userId'].apply(lambda x: userid2idx[x])
df['movieId'] = df['movieId'].apply(lambda x: movieid2idx[x])
split = np.random.rand(len(df)) < 0.8
train = df[split]
valid = df[~split]
print(train.shape , valid.shape)

(16000220, 4) (4000043, 4)


In [246]:
user_encoder= LabelEncoder()
movie_encoder= LabelEncoder()
train['userId']= user_encoder.fit_transform(train['userId'])
train['movieId']=movie_encoder.fit_transform(train['movieId'])
valid['userId']= user_encoder.fit_transform(valid['userId'])
valid['movieId']=movie_encoder.fit_transform(valid['movieId'])

<ipython-input-246-d8f699dd06d9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['userId']= user_encoder.fit_transform(train['userId'])
<ipython-input-246-d8f699dd06d9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['movieId']=movie_encoder.fit_transform(train['movieId'])
<ipython-input-246-d8f699dd06d9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [247]:
df.head()

,userId,movieId,rating,timestamp
0,0,0,3.5,2005-04-02 23:53:47
1,0,1,3.5,2005-04-02 23:31:16
2,0,2,3.5,2005-04-02 23:33:39
3,0,3,3.5,2005-04-02 23:32:07
4,0,4,3.5,2005-04-02 23:29:40


In [248]:
df.userId.nunique(),df.movieId.nunique()

(138493, 26744)

In [249]:
u_train= train['userId']
m_train= train['movieId']
r_train= train['rating']
lu_train= u_train.nunique()
lm_train= m_train.nunique()

In [250]:
import sys
print("Original train size:",sys.getsizeof(train))


Original train size: 1728023776


In [251]:
matrix_train= coo_matrix((r_train,(u_train,m_train)), shape=(lu_train, lm_train))

In [252]:
print("Coordinate matrix size: ", sys.getsizeof(matrix))

Coordinate matrix size:  48


In [253]:
u_val= valid['userId']
m_val= valid['movieId']
r_val= valid['rating']
lu_val= u_val.nunique()
lm_val= m_val.nunique()

In [254]:
matrix_val= coo_matrix((r_val,(u_val,m_val)), shape=(lu_train, lm_train))

In [255]:
model= LightFM(no_components=30, k=5, learning_rate=0.05, random_state=33)


In [256]:
model.fit(matrix_train,epochs= 50, num_threads= 4)

In [257]:
matrix

<13849x16456 sparse matrix of type '<class 'numpy.float32'>'
	with 1613387 stored elements in COOrdinate format>

In [258]:
from lightfm.evaluation import auc_score, precision_at_k
print("Train AUC Score::",auc_score(model, matrix_train, num_threads=4).mean())
print("Test AUC Score:",auc_score(model, matrix_val, num_threads=4).mean())


Train AUC Score:: 0.9694958
Test AUC Score: 0.9507404
